In [1]:
import json
import numpy as np
import pandas as pd
from IPython.display import JSON

In [2]:
def read_raw(filename, subdir='raw'):
    """Reads the raw json data file @filename and returns it as a dict"""
    with open(f'../data/{subdir}/{filename}.json') as file:
        return json.load(file)
    
def write_processed_df(filename, dataframe, orient='index'):
    """Writes the @dataframe to a processed json data file @filename"""
    dataframe.to_json(f'../data/processed/{filename}.json', orient=orient, indent=4)

def write_processed_dict(filename, dictionary):
    """Writes the @dictionary to a processed json data file @filename"""
    with open(f'../data/processed/{filename}.json', 'w') as json_file:
        json.dump(dictionary, json_file, indent=4)

In [3]:
epfl = read_raw('epfl')
epfl_masterspecs = read_raw('epfl_master_specs')
epfl_studyplans = read_raw('epfl_studyplans')
isa = read_raw('isa_registrations')

Snapshots from the json objects:

In [4]:
JSON(epfl['master']['Data Science']['COM-480'])

<IPython.core.display.JSON object>

In [5]:
JSON(epfl_masterspecs['Civil Engineering'])

<IPython.core.display.JSON object>

In [6]:
epfl_studyplans['master']['Data Science']

'/studyplan/en/master/data-science'

In [7]:
JSON(isa['Data visualization'])

<IPython.core.display.JSON object>

Let's start with noting that only 25 master's programs are currently offered at EPFL and only 13 bachelor programs. So, we have some outdated programs in our database. Let's remove them.

In [8]:
{level: len(programs) for level, programs in epfl.items()}

{'propedeutics': 14,
 'bachelor': 17,
 'master': 29,
 'minor': 20,
 'doctoral_school': 22}

In [9]:
# Only 13 bachelor programs are currently offered
# + Humanities and Social Sciences Program
# + Design Together ENAC
# https://www.epfl.ch/education/bachelor/programs/
bachelor_not_current = ['Chemistry', 'Chemical Engineering']

# Only 25 master's programs are currently offered
# + Humanities and Social Sciences Program
# https://www.epfl.ch/education/master/programs/
master_not_current = [
    'Bioengineering',
    'Life Sciences and Technologies - master program',
    'Micro- and Nanotechnologies for Integrated Systems'
]

# The following minors are missing, among possibly others:
#   Computational science and engineering
#   Mechanical engineering
# We do not take any action for now

# we can concatenate the lists because these program names are unique across all levels
# note: we can't use this concatenation below if the condition breaks, and need to distinguish between levels
not_current = bachelor_not_current + master_not_current

In [10]:
# remove programs not currently offered, everywhere they occur
for level_name in list(epfl):
    for program_name in list(epfl[level_name]):
        if program_name in not_current:
            del epfl[level_name][program_name]
            del epfl_studyplans[level_name][program_name]
            if level_name == 'master' and program_name in epfl_masterspecs:
                # remove program from the list of programs with specializations
                del epfl_masterspecs[program_name]
                
        # remove entries from `in_the_programs` field
        if program_name in epfl[level_name]:
            for course_code in list(epfl[level_name][program_name]):
                in_the_programs = epfl[level_name][program_name][course_code]['in_the_programs']
                # in-place modification
                in_the_programs[:] = (item for item in in_the_programs if item[0] not in not_current)

In [11]:
write_processed_dict('epfl_studyplans', epfl_studyplans)

Let's now analyze master's specializations (no specializations at other levels) and evaluate data accuracy. We compare the data source (studyplan pages) with the studyplan brochures. We have created a separate json object `epfl_master_specs` to store information about specializations.

In [12]:
"""
The studyplan page sometimes differs greatly from the up-to-date studyplan brochure
which might indicate that the studyplan page hasn't been updated.
We skip these specializations for now (we would have to manually type them in):

Architecture
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/ENAC_ARCHI_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/architecture
"""
specs_to_remove = ['Architecture']

"""
The specializations legend on studyplan pages is sometimes obsolete (hasn't been removed)
We remove these programs from the list of specialization programs

Materials Science and Engineering (only one specialization now)
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_MX_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/materials-science-and-engineering

Microengineering
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_SMT_MA_RV-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/microengineering
"""
specs_to_remove.extend(['Materials Science and Engineering', 'Microengineering'])

"""
The specializations legend on studyplan pages is sometimes obsolete, but the studyplan itself
contains references to specializations that correctly correspond to the the studyplan brochure
In this case, we update the legend manually and fix the data

Electrical and Electronics Engineering
https://www.epfl.ch/education/master/wp-content/uploads/2018/08/STI_EL_MA-1.pdf
vs.
https://edu.epfl.ch/studyplan/en/master/electrical-and-electronics-engineering
"""
electrical_electronics_eng_specs = {
    "a": "Microelectronics circuits and systems",
    "b": "Electronic technologies and device-circuit interactions",
    "c": "Bioelectronics",
    "d": "Internet of Things (IoT)",
    "e": "Data Science and Systems",
    "f": "Signal, Image, Video and Communication",
    "g": "Wireless and Photonics Circuits and Systems",
}

We remove all programs in `specs_to_remove`:

In [13]:
for program in specs_to_remove:
    del epfl_masterspecs[program]

... and fix "Electrical and Electronics Engineering" specializations key:

In [14]:
epfl_masterspecs['Electrical and Electronics Engineering']['spec_key'] = electrical_electronics_eng_specs

We observe that some courses have listed specializations not present in the `spec_key` for the given program. We remove these references:

In [15]:
for program_name, program_spec in epfl_masterspecs.items():
    for code, course_spec in program_spec['courses'].items():
        if (any(spec not in program_spec['spec_key'] for spec in course_spec)):
            print(f"{program_name}, spec key: {list(program_spec['spec_key'].keys())}, {code}: {course_spec}")
            
            new_course_spec = [spec for spec in course_spec if spec in program_spec['spec_key']]
            epfl_masterspecs[program_name]['courses'][code] = new_course_spec
            
            print(f"Course spec changed from {course_spec} to {new_course_spec}")

Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-588: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], MATH-261: ['d', 'e', 'h']
Course spec changed from ['d', 'e', 'h'] to ['d', 'e']
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], CS-423: ['d', 'f', 'h']
Course spec changed from ['d', 'f', 'h'] to ['d', 'f']
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-576: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], ME-409: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], EE-466: ['h']
Course spec changed from ['h'] to []
Electrical and Electronics Engineering, spec key: ['a', 'b', 'c', 'd', 'e', 'f', 'g'], MICRO-565: ['h']
Course spe

Next, we add a `specs` dictionary to each program in `epfl_masterspecs` with keys and values switching roles compared to the `courses` property. This will make it easier to look up all courses with a given specialization.

In [16]:
# courses property
list(epfl_masterspecs['Computer Science']['courses'].items())[:10]

[('CS-595', []),
 ('CS-599', []),
 ('CS-450', ['b', 'c', 'd', 'e', 'i']),
 ('CS-470', ['a', 'd', 'g']),
 ('CS-442', ['f']),
 ('COM-401', ['d', 'e', 'g', 'j']),
 ('CS-422', ['b', 'c', 'g', 'j']),
 ('CS-438', ['g']),
 ('CS-451', ['c', 'g', 'i', 'j']),
 ('CS-452', ['c', 'g'])]

In [17]:
for program_name, program_spec in epfl_masterspecs.items():
    # initialize dictionary with empty list values
    epfl_masterspecs[program_name]['specs'] = {k: [] for k in epfl_masterspecs[program_name]['spec_key']}
    
    for course_code, course_spec in program_spec['courses'].items():
        # iterate over all specialization codes for each course, and add the course code to the specs dictionary
        for k in course_spec:
            epfl_masterspecs[program_name]['specs'][k].append(course_code)

In [18]:
# specs property
{'g': epfl_masterspecs['Computer Science']['specs']['g']}

{'g': ['CS-470',
  'COM-401',
  'CS-422',
  'CS-438',
  'CS-451',
  'CS-452',
  'COM-407',
  'CS-420',
  'CS-471',
  'CS-453',
  'COM-405',
  'COM-503',
  'CS-522']}

In [19]:
write_processed_dict('epfl_master_specs', epfl_masterspecs)

Now, we have processed the specializations and can move on.

Next step is to create two new json objects by transforming the `epfl` object. The goal will be to remove redundancies in the original object, since it has many courses occuring repeatedly, and not only enable easy access to programs, but also _individual courses_. The idea is to create one object `epfl_courses` with all course details and another object `epfl_programs` which will only include course codes for each program.

In [20]:
epfl_courses = [
    # for each program: ([code1, code2, ...], [course1, course2, ...])
    ([*program.keys()], [*program.values()])
    for level_name, level in epfl.items()
    for program_name, program in level.items()
]

In [21]:
# all courses: [[code1, code2, ...], [course1, course2, ...]]
epfl_courses = list(
    map(
        lambda x: sum(x, []),
        zip(*epfl_courses)
    )
)

Let's analyze the list of courses for all programs. Let's use pandas to help us with that:

In [22]:
courses_df = pd.DataFrame(index=epfl_courses[0], data=epfl_courses[1])

In [23]:
courses_df.columns

Index(['name', 'section', 'language', 'note', 'path', 'semester', 'exam_form',
       'coefficient', 'subject_examined', 'lecture', 'exercises', 'lecturers',
       'required', 'recommended', 'concepts', 'in_the_programs',
       'prerequisite_for', 'project', 'number_of_places', 'practical_work',
       'credits', '', 'labs'],
      dtype='object')

We observe there are a couple of fields we don't recognize, `coefficient` and `''`. Let's analyze those first.

In [24]:
# the '' column has all values either as NaN or '', so we can safely delete the column
courses_df[''].unique()

array([nan, ''], dtype=object)

In [25]:
courses_df.drop('', axis=1, inplace=True)

We observed that the _five_ rows having `''` field as the empty string are exactly the ones that don't have the number of credits listed, but we shall fix this soon (corresponding html element is empty).

The `coefficient` column is an alias for `credits`

In [26]:
len(courses_df)

3443

In [27]:
courses_df.coefficient.isna().sum()

3205

In [28]:
courses_df.credits.isna().sum()

243

In [29]:
(courses_df.credits == courses_df.coefficient).any()

False

No rows have both `credits` and `coefficient`, and there are 5 rows which have neither. Let's first merge the columns and then manually fill in the 5 gaps if possible

In [30]:
courses_df.credits.fillna(courses_df.coefficient, inplace=True)
courses_df.drop('coefficient', axis=1, inplace=True)

Let's now see the reason behind the 5 gaps:

In [31]:
courses_df[courses_df.credits.isna()].index

Index(['ENG-274', 'CH-361', 'CH-361', 'PENS-200', 'PHYS-300(a)'], dtype='object')

We can safely fill the gaps with 0 credits since:
* ENG-274 is without credits
* CH-361 is an optional course, pointing to one of the advanced general chemistry courses
* PENS-200 Ground control in Swiss law, credits are included in the ENAC week
* PHYS-300(a) is also without credits

In [32]:
courses_df.fillna(value={'credits': '0'}, inplace=True)

Let's also remove the field `subject_examined` since in most cases it contains the same information as the `name` field, and the name column is always more specific and doesn't contain any gaps.

In [33]:
courses_df[['name', 'subject_examined']][~(courses_df['name'] == courses_df.subject_examined)].head()

,name,subject_examined
PHYS-118,Building physics,Building physics
MATH-189,Mathematics,Mathematics
AR-113,Graphic figuration\n,Graphic figuration
AR-101,Studio BA1,"Studio BA1, BA2"
AR-102,Studio BA2,"Studio BA1, BA2"


In [34]:
courses_df.drop('subject_examined', axis=1, inplace=True)

We know that courses are repeated in the dataframe (row indices), but we need to check if the information is the same when course codes match. It is likely that they are the same, but we can't be sure because of the way the coursebooks were harvested (using program-specific query string parameters for fields in the right column of the page).

In [35]:
f"{courses_df.index.nunique()} unique courses and {len(courses_df)} courses with duplicates"

'1886 unique courses and 3443 courses with duplicates'

Before moving on, we clean the coursebook paths by removing query parameters (`cb_cycle` and `cb_section`) from them, since they are different for each level and program and don't seem to provide additional information.

In [36]:
courses_df['path'] = courses_df.path.str.split('?').str[0]

Let's see if we have a unique row count of 1 for each course code:

In [37]:
# Convert list columns to tuple to avoid this error when calling nunique():
# TypeError: unhashable type: 'list'
list_columns = ['lecturers', 'required', 'recommended', 'concepts', 'prerequisite_for', 'in_the_programs']

In [38]:
courses_df[list_columns] = courses_df[list_columns].applymap(tuple)
# lecturers and in_the_programs field is a list of lists
courses_df['lecturers'] = courses_df.lecturers.apply(lambda x: tuple(map(tuple, x)))
courses_df['in_the_programs'] = courses_df.in_the_programs.apply(lambda x: tuple(map(tuple, x)))

In [39]:
# path is still sometimes different, because of alias paths which point to the same coursebook (although the pages might differ slightly).
# We will not make sure paths point to pages with correct information unless there are
# inconsistencies and one path is more up-to-date than others.
group = courses_df.drop('path', axis=1).groupby(courses_df.index)

In [40]:
exceptions = group.nunique().loc[(group.nunique() > 1).any(axis=1)]
exceptions[exceptions.columns[exceptions.gt(1).any()]]

,note,semester,lecturers,in_the_programs
BIO-502,1,1,1,2
BIOENG-404,1,1,1,2
CH-361,2,1,1,2
COM-208,1,1,1,2
COM-480,1,2,1,1
CS-433,2,1,1,1
EE-111,2,2,1,1
EE-492(d),2,1,1,2
EE-727,1,2,1,1
ENG-704,2,1,1,1


In [41]:
from operator import add

courses_df.loc['BIO-502', 'path'] = courses_df.loc['BIO-502', 'path'][1]
courses_df.loc['BIO-502', 'in_the_programs'] = [courses_df.loc['BIO-502', 'in_the_programs'][1]] * 2

courses_df.loc['BIOENG-404', 'path'] = courses_df.loc['BIOENG-404', 'path'][0]
courses_df.loc['BIOENG-404', 'in_the_programs'] = [add(*courses_df.loc['BIOENG-404', 'in_the_programs'][1:3])] * 3

courses_df.loc['CH-361', 'path'] = courses_df.loc['CH-361', 'path'][1]
courses_df.loc['CH-361', 'in_the_programs'] = [add(*courses_df.loc['CH-361', 'in_the_programs'])] * 2
courses_df.loc['CH-361', 'note'] = courses_df.loc['CH-361', 'note'][0]  # the content of the notes are the same, just written differently

courses_df.loc['COM-208', 'path'] = courses_df.loc['COM-208', 'path'][0]
courses_df.loc['COM-208', 'in_the_programs'] = [add(*courses_df.loc['COM-208', 'in_the_programs'][3:5])] * 5

courses_df.loc['COM-480', 'semester'] = courses_df.loc['COM-480', 'semester'][0]

courses_df.loc['CS-401', 'path'] = courses_df.loc['CS-401', 'path'][0]
courses_df.loc['CS-401', 'in_the_programs'] = [courses_df.loc['CS-401', 'in_the_programs'][1]] * 16

courses_df.loc['CS-433', 'note'] = courses_df.loc['CS-433', 'note'][0]

courses_df.loc['EE-111', 'semester'] = 'Fall/Spring'
courses_df.loc['EE-111', 'note'] = courses_df.loc['EE-111', 'note'][1]

courses_df.loc['EE-492(d)', 'in_the_programs'] = [add(*courses_df.loc['EE-492(d)', 'in_the_programs'])[::2]] * 2  # semester == 'Fall'
courses_df.loc['EE-492(d)', 'note'] = courses_df.loc['EE-492(d)', 'note'][0]

courses_df.loc['EE-727', 'semester'] = courses_df.loc['EE-727', 'semester'][0]

courses_df.loc['ENG-704', 'note'] = courses_df.loc['ENG-704', 'note'][0]

courses_df.loc['ENV-342', 'in_the_programs'] = [courses_df.loc['ENV-342', 'in_the_programs'][0]] * 2
courses_df.loc['ENV-342', 'path'] = courses_df.loc['ENV-342', 'path'][1]
courses_df.loc['ENV-342', 'lecturers'] = [courses_df.loc['ENV-342', 'lecturers'][1]] * 2

courses_df.loc['ENV-510', 'semester'] = courses_df.loc['ENV-510', 'semester'][0]

courses_df.loc['MATH-498', 'in_the_programs'] = [add(*courses_df.loc['MATH-498', 'in_the_programs'])] * 2

courses_df.loc['MATH-599', 'in_the_programs'] = [add(*courses_df.loc['MATH-599', 'in_the_programs'])] * 2

courses_df.loc['ME-467', 'semester'] = courses_df.loc['ME-467', 'semester'][0]

courses_df.loc['MGT-418', 'note'] = ''  # unclear if note applies anyway

courses_df.loc['MICRO-452', 'path'] = courses_df.loc['MICRO-452', 'path'][0]
courses_df.loc['MICRO-452', 'in_the_programs'] = [courses_df.loc['MICRO-452', 'in_the_programs'][0]] * 2

courses_df.loc['MICRO-455', 'path'] = courses_df.loc['MICRO-455', 'path'][3]
courses_df.loc['MICRO-455', 'in_the_programs'] = [courses_df.loc['MICRO-455', 'in_the_programs'][3]] * 8
courses_df.loc['MICRO-455', 'lecturers'] = [courses_df.loc['MICRO-455', 'lecturers'][3]] * 8

courses_df.loc['MICRO-513', 'path'] = courses_df.loc['MICRO-513', 'path'][1]
courses_df.loc['MICRO-513', 'in_the_programs'] = [courses_df.loc['MICRO-513', 'in_the_programs'][1]] * 7

courses_df.loc['MICRO-597', 'path'] = courses_df.loc['MICRO-597', 'path'][0]
courses_df.loc['MICRO-597', 'in_the_programs'] = [tuple(sorted(courses_df.loc['MICRO-597', 'in_the_programs'][0] + (('Robotics', 'Master Project autumn'),)))] * 2

courses_df.loc['MICRO-723', 'semester'] = courses_df.loc['MICRO-723', 'semester'][0]

courses_df.loc['PENS-491', 'in_the_programs'] = [add(*courses_df.loc['PENS-491', 'in_the_programs'][1:3])] * 5

courses_df.loc['PHYS-201(e)', 'in_the_programs'] = [add(*courses_df.loc['PHYS-201(e)', 'in_the_programs'])] * 2

Now we can remove duplicate rows/indices from the dataframe

In [42]:
courses_unique_df = courses_df.loc[~courses_df.index.duplicated()]
len(courses_unique_df)

1886

Strip whitespace and newline characters from all strings

In [43]:
courses_unique_df = courses_unique_df.applymap(lambda v: v.strip().replace('\xa0', ' ') if type(v) == str else v)

While dealing with the exceptions, we noticed that some courses might be given during both Spring and Fall semesters. Most of these must be internships and master's projects. Let's check them out and fix the `semester` field for those.

In [44]:
def filter_fall_spring():
    in_programs = courses_unique_df.in_the_programs
    semester_str = in_programs.apply(lambda r: ''.join([i[1] for i in r]))
    sem_fall_spring_idx = semester_str.index[semester_str.map(lambda sem: any(w in sem for w in ['fall', 'autumn']) and 'spring' in sem)]
    return sem_fall_spring_idx


fall_spring_idx = filter_fall_spring()
courses_unique_df.loc[fall_spring_idx][['name', 'semester', 'credits']]

,name,semester,credits
MATH-597,Engineering internship credited with master pr...,Fall,0
MATH-599,Master project in Mathematics,Fall,30
PHYS-599(a),Master project in Physics Engineering,Fall,30
AR-599,Master project in Architecture,Fall,30
ChE-599,Master project in Chemical Engineering & Biote...,Fall,30
CIVIL-597,Engineering internship (master in Civil engine...,Fall,0
CIVIL-599,Master project in Civil Engineering,Fall,30
COM-597,Engineering internship credited with Master pr...,Fall,0
COM-599,Master project in Communication Systems,Fall,30
MATH-598,Projet de master en science et ingénierie comp...,Fall,30


We observe that all of those are Master's projects or internships. Let's set the `semester` as Fall/Spring for all of them.

In [45]:
courses_unique_df.loc[fall_spring_idx, 'semester'] = 'Fall/Spring'

In [46]:
# Rest of the courses have unspecified semester info, in most cases those must be doctoral courses
courses_unique_df.loc[courses_unique_df.semester == '', 'semester'] = 'Inapplicable'

***
Next step is to process the registration data and integrate it with `courses_unique_df`. We will also store it in separate json files.

In [47]:
# from_dict disregards keys with empty dictionaries as value
isa_df = pd.DataFrame.from_dict(isa, orient='index')

In [48]:
sum(value == {} for value in isa.values())

194

In [49]:
isa_df.head(1)

,2018-2019,2015-2016,2013-2014,2017-2018,2012-2013,2016-2017,2014-2015,2011-2012,2019-2020,2010-2011,2008-2009,2009-2010,2007-2008,2006-2007,2005-2006,2004-2005,2020-2021
Numerical approximation of PDE's II,"{'Applied Mathematics, Master semester 1, 3': ...","{'Applied Mathematics, Master semester 2, 4': ...","{'Applied Mathematics, Master semester 2, 4': ...","{'Computational science and Engineering, Maste...","{'Applied Mathematics, Master semester 2': 11,...","{'Mathematics - master program, Master semeste...","{'Applied Mathematics, Master semester 2, 4': ...",NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


Let's exchange the course name with the course code from `courses_unique_df`. This will drop unmatched courses from `isa_df`:

In [50]:
isa_df = (
    courses_unique_df.join(isa_df, on='name', how='inner')
    .drop(courses_unique_df.columns, axis=1)
)

In [51]:
# courses with no registration data
courses_unique_df.index[~courses_unique_df.index.isin(isa_df.index)]

Index(['CH-315', 'CH-361', 'HUM-383', 'MATH-476', 'AR-468', 'CH-428',
       'CIVIL-422', 'DH-490', 'EE-520', 'AR-403', 'ME-468', 'CH-418',
       'ETH-427', 'ETH-444', 'ETH-453', 'ETH-450', 'ETH-451', 'AR-804',
       'ChE-607(1)', 'ChE-607(2)', 'ChE-608(1)', 'ChE-608(2)', 'MATH-801(2)',
       'CS-803', 'ENG-627', 'ENG-637', 'ENG-800', 'ENG-630', 'MGT-641(a)',
       'MGT-641(b)', 'MGT-642(a)', 'EE-609', 'MGT-693(E)', 'MGT-693(A)',
       'MGT-693(I)', 'MSE-655', 'MSE-667', 'MSE-715', 'MSE-603', 'MATH-620(2)',
       'ME-628', 'BIO-603(MS)', 'BIO-699(n)', 'PHYS-639'],
      dtype='object')

In [52]:
# sort columns, academic years, in descending order
isa_df = isa_df[isa_df.columns.sort_values()]

In [53]:
isa_df.head()

,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021
AR-126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 2': 136, 'to...","{'Architecture, Bachelor semester 2': 147, 'to...",NaN
MATH-126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Passerelle HES - AR, Spring semester': 1, 'A...","{'Passerelle HES - AR, Spring semester': 2, 'A...","{'Architecture, Bachelor semester 2': 240, 'Pa...","{'Passerelle HES - AR, Spring semester': 5, 'A...","{'Passerelle HES - AR, Spring semester': 5, 'A...","{'Passerelle HES - AR, Spring semester': 3, 'A...",NaN
CIVIL-123,"{'Environmental Sciences and Engineering, Bach...","{'Environmental Sciences and Engineering, Bach...","{'Civil Engineering, Bachelor semester 2': 75,...","{'Civil Engineering, Bachelor semester 2': 69,...","{'Civil Engineering, Bachelor semester 2': 77,...","{'Architecture, Bachelor semester 2': 265, 'Ci...","{'Environmental Sciences and Engineering, Bach...","{'Civil Engineering, Bachelor semester 2': 174...","{'Civil Engineering, Bachelor semester 2': 178...","{'Civil Engineering, Bachelor semester 2': 157...","{'Architecture, Bachelor semester 2': 207, 'Ci...","{'Architecture, Bachelor semester 2': 217, 'Ci...","{'Architecture, Bachelor semester 2': 176, 'Ci...","{'Architecture, Bachelor semester 2': 166, 'Ci...","{'Architecture, Bachelor semester 2': 137, 'Ci...","{'Civil Engineering, Bachelor semester 2': 59,...",NaN
PHYS-118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 1': 218, 'to...","{'Architecture, Bachelor semester 1': 234, 'to...",NaN
AR-125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 1': 202, 'to...","{'Architecture, Bachelor semester 1': 236, 'to...",NaN


Next we create a new dataframe for total registrations per course and per year, `isa_total_df`:

In [54]:
isa_total_df = (
    isa_df.stack()     # stack year columns on the index
    .apply(pd.Series)  # expand each dictionary value into a Series
    .total             # select the 'total' column
    .unstack(level=1)  # unstack the year index level to the column axis
    .fillna(0)         # fill NaN totals with 0
    .astype(int)       # float --> int
)

In [55]:
isa_total_df.head()

,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021
AR-126,0,0,0,0,0,0,0,0,0,0,0,0,0,0,136,147,0
MATH-126,0,0,0,0,0,0,0,0,0,0,277,258,249,171,145,151,0
CIVIL-123,255,272,237,208,258,373,426,437,428,388,309,317,231,232,197,206,0
PHYS-118,0,0,0,0,0,0,0,0,0,0,0,0,0,0,218,234,0
AR-125,0,0,0,0,0,0,0,0,0,0,0,0,0,0,202,236,0


We add `registrations` column to `courses_unique_df`

In [56]:
isa_total_df_compressed = isa_total_df.apply(dict, axis=1).to_frame('registrations')
isa_total_df_compressed.head()

,registrations
AR-126,"{'2004-2005': 0, '2005-2006': 0, '2006-2007': ..."
MATH-126,"{'2004-2005': 0, '2005-2006': 0, '2006-2007': ..."
CIVIL-123,"{'2004-2005': 255, '2005-2006': 272, '2006-200..."
PHYS-118,"{'2004-2005': 0, '2005-2006': 0, '2006-2007': ..."
AR-125,"{'2004-2005': 0, '2005-2006': 0, '2006-2007': ..."


In [57]:
courses_unique_df = courses_unique_df.join(isa_total_df_compressed, how='outer')

Before moving on we remove certain columns from `course_unique_df` which contain raw info we will process later to extract links between courses. We store these in a new frame, `dependency_info_df`:

In [58]:
dependency_info_cols = ['required', 'recommended', 'concepts', 'prerequisite_for']

dependency_info_df = courses_unique_df[dependency_info_cols]
courses_unique_df.drop(dependency_info_cols, axis=1, inplace=True)

Now we can write the dataframes to json files.

In [59]:
isa_dict = isa_df.to_dict(orient='index')

In [60]:
# create dictionary with NaNs removed
isa_dict = {
    code: {
        year: info
        for year, info in course.items()
        if type(info) == dict  # skip NaN
    }
    for code, course in isa_dict.items()
}

In [61]:
write_processed_df('isa_all', isa_df)
write_processed_dict('isa_all_notnan', isa_dict)

write_processed_df('isa_total', isa_total_df)
write_processed_df('epfl_courses', courses_unique_df)

In [62]:
isa_df

,2004-2005,2005-2006,2006-2007,2007-2008,2008-2009,2009-2010,2010-2011,2011-2012,2012-2013,2013-2014,2014-2015,2015-2016,2016-2017,2017-2018,2018-2019,2019-2020,2020-2021
AR-126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 2': 136, 'to...","{'Architecture, Bachelor semester 2': 147, 'to...",NaN
MATH-126,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Passerelle HES - AR, Spring semester': 1, 'A...","{'Passerelle HES - AR, Spring semester': 2, 'A...","{'Architecture, Bachelor semester 2': 240, 'Pa...","{'Passerelle HES - AR, Spring semester': 5, 'A...","{'Passerelle HES - AR, Spring semester': 5, 'A...","{'Passerelle HES - AR, Spring semester': 3, 'A...",NaN
CIVIL-123,"{'Environmental Sciences and Engineering, Bach...","{'Environmental Sciences and Engineering, Bach...","{'Civil Engineering, Bachelor semester 2': 75,...","{'Civil Engineering, Bachelor semester 2': 69,...","{'Civil Engineering, Bachelor semester 2': 77,...","{'Architecture, Bachelor semester 2': 265, 'Ci...","{'Environmental Sciences and Engineering, Bach...","{'Civil Engineering, Bachelor semester 2': 174...","{'Civil Engineering, Bachelor semester 2': 178...","{'Civil Engineering, Bachelor semester 2': 157...","{'Architecture, Bachelor semester 2': 207, 'Ci...","{'Architecture, Bachelor semester 2': 217, 'Ci...","{'Architecture, Bachelor semester 2': 176, 'Ci...","{'Architecture, Bachelor semester 2': 166, 'Ci...","{'Architecture, Bachelor semester 2': 137, 'Ci...","{'Civil Engineering, Bachelor semester 2': 59,...",NaN
PHYS-118,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 1': 218, 'to...","{'Architecture, Bachelor semester 1': 234, 'to...",NaN
AR-125,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'Architecture, Bachelor semester 1': 202, 'to...","{'Architecture, Bachelor semester 1': 236, 'to...",NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
PHYS-810,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'total': 24, 'Physics (edoc)': 24}",NaN
PHYS-812,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'total': 24, 'Physics (edoc)': 24}",NaN
PHYS-808,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'total': 95, 'Physics (edoc)': 95}",NaN
PHYS-809,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,"{'total': 31, 'Physics (edoc)': 31}",NaN


Create `epfl_programs` dict and write to json file:

In [63]:
# use dict comprehension to produce epfl_programs
epfl_programs = {
    level_name: {
        program_name: list(program.keys())
        for program_name, program in level.items()
    }
    for level_name, level in epfl.items()
}

write_processed_dict('epfl_programs', epfl_programs)

In [64]:
epfl_programs['bachelor'].keys()

dict_keys(['Architecture', 'Chemistry and Chemical Engineering', 'Civil Engineering', 'Communication Systems', 'Computer Science', 'Design Together ENAC', 'Electrical and Electronics Engineering', 'Environmental Sciences and Engineering', 'Humanities and Social Sciences Program', 'Life Sciences Engineering', 'Materials Science and Engineering', 'Mathematics', 'Mechanical Engineering', 'Microengineering', 'Physics'])

In [65]:
epfl_programs['bachelor']['Life Sciences Engineering'][:10]

['MATH-203(a)',
 'MATH-207(a)',
 'PHYS-201(a)',
 'PHYS-207(a)',
 'MATH-251(c)',
 'MATH-231',
 'MATH-236',
 'BIO-212',
 'BIO-213',
 'BIO-205']

***
Now it's time to construct the adjacency list for the course links:

In [66]:
dependency_info_df

,required,recommended,concepts,prerequisite_for
AR-101,(),(),(),()
AR-102,(),(),(),()
AR-113,"(Aucun prérequis,)",(),(),()
AR-121,(),(),"(Modernité architecturale, plasticité, structu...",()
AR-122,(),(),(),()
...,...,...,...,...
PHYS-808,(),(),(),()
PHYS-809,(),(),(),()
PHYS-810,(),(),(),()
PHYS-811,"(Condensed matter physics, quantum physics,)",(),(),()


In [67]:
# create a masterfile
processed_programs = read_raw('epfl_programs', subdir='processed')
processed_studyplans = read_raw('epfl_studyplans', subdir='processed')
processed_courses = read_raw('epfl_courses', subdir='processed')
processed_specs = read_raw('epfl_master_specs', subdir='processed')

write_processed_dict('master', {
    'programs': processed_programs,
    'studyplans': processed_studyplans,
    'courses': processed_courses,
    'masterspecs': processed_specs
})